# 1. 컨셉

* 카메라로부터의 원근 정도에 따라 픽셀 수가 동일하더라도 그에 대한 실제 거리는 다를 것이다.  
* 그 변화하는 정도를 알 수 있다면, 픽셀에 대한 x,y 좌표값을 실제 거리로 환산 가능할 것이다.  
* 실제 거리로 환산한 후, object와 카메라를 동일 3차원 좌표계에 위치시킨다.  
* object는 지상에 있으므로 높이가 0이라고 가정, 그 좌표가 (x, y, 0)이 된다.  
* 한편, 카메라의 좌표는 (0, -(거리의 폭), (카메라의 높이))가 된다.  
  (참고: 카메라의 x축 위치는 이미지의 가로 길이의 중점과 같고, y축 위치는 이미지 최하단에서 도로의 폭만큼 떨어져 있으며, z축 위치는 건물 8층 높이다)  
* 둘의 좌표에 유클리디안 거리를 적용하여 둘 간의 거리를 구할 수 있을 것이다.  

# 2. 절차

#### 1) 수식 만들기  
#### 2) 실제 데이터 수집  
#### 3) 대입하여 수식 내 계수 구하기  

# 3. 절차별 상세 내용

### 1) 수식 만들기

* 픽셀의 x,y값이 있을 때, 각각에 scale factor를 곱한 후 bias를 더하여 실제 거리를 구하는 수식을 만들 수 있다.  
이 때 scale factor ($s_y$)는 이미지에서 세로 방향으로 멀어짐에 따른 원근을 반영하기 위한 것으로, 픽셀의 y축 값에 대해 영향을 받는다.  
이를 각각 1차식으로 표현하면 아래와 같다.

$s_y = a_y * p_y + b_y$  ... 1)    
$x = s_y * p_x + b_m$  ... 2)  
$y = s_y * p_y + b_n$  (= $a_y*p_y^2 + b_y*p_y + b_n$)  ...3)

($s_y$: scale factor로, 픽셀의 y축 값 ($p_y$)에 대한 일차식의 형태이다.  
 $x, y$: 미터 단위로 환산된 실제 x축, y축 방향의 거리이다.  
 $p_x, p_y$: 이미지 최하단의 중점을 원점으로 하는 좌표계로 변환된 x,y의 픽셀값이며, 0에서 1 사이의 값을 가진다.  
 $b_m, b_n$: bias 값)

* 위의 식에서 구해야 하는 값은 $a_y, b_y, b_m, b_n$ 이다.
* 그런데 이미지의 원점 (($p_x, p_y$) = (0, 0))에서는 실제 거리도 0이므로 $b_m$=$b_n$=0 이다.  
* 따라서 $a_y$와 $b_y$ 두 개의 값만 구하면 되며, 이를 위해 3개의 물체에 대해 픽셀 거리와 실제 거리를 실측하였다.  
* 도로 (이미지와 카메라 사이에 있음)의 폭과 카메라의 높이 또한 실측하였다.  
  (레이저 측정기 사용)

### 2) 실제 데이터 수집

* 실측 결과, 3개 물체의 픽셀 좌표 및 실제 거리는 아래와 같았다.  
  (pixels의 각 원소는 각 물체 양 끝점의 y좌표로, 둘의 차이는 물체의 y축 방향으로의 크기와 동일하다. 그리고 그 각각의 실제 길이는 real_distances의 원소와 대응된다.)

    - pixels = [[0.2166665, 0.3374995], [0.5819445, 0.6611115], [0.877778, 0.902778]]
    - real_distances = [5.21, 7.244, 4.991]  
    
    
* 한편, 도로의 폭과 카메라의 높이는 아래와 같았다. (미터 단위)

    - 도로의 폭 = 16.579  
    - 카메라의 높이 = 24.749 


### 3) 대입하여 수식 내 계수 구하기

* 리스트 pixels의 각 원소들 내 두 값을 수식 3)에 대입한 후 그 차이를 구하는 세 개의 식을 만들 수 있다.  
* 만들어진 세 개의 식으로부터 유사 역행렬 계산을 통하여 행렬 v ([$a_y, b_y$])를 구할 수 있으며, 그 과정은 아래와 같다.

```
Av = b
```
(A: 위 세 개의 식을 통해 얻은 값들의 행렬로, 3\*2 행렬이다.  
 v: $a_y, b_y$를 원소로 가지며, 2\*1 행렬이다.  
 b: 3개의 실제 거리값으로 이루어져 있으며, 3\*1 행렬이다)

In [9]:
import numpy as np

# 계산 과정
pixels = [[0.2166665, 0.3374995], [0.5819445, 0.6611115], [0.877778, 0.902778]]
real_distances = [5.21, 7.244, 4.991]

# A 구하기
A = ([pixels[0][1]**2 - pixels[0][0]**2, pixels[0][1] - pixels[0][0]],
     [pixels[1][1]**2 - pixels[1][0]**2, pixels[1][1] - pixels[1][0]],
     [pixels[2][1]**2 - pixels[2][0]**2, pixels[2][1] - pixels[2][0]])
A = np.array(A)

# b 구하기
b = real_distances

# A_inv (유사 역행렬) 구하기
A_inv = np.linalg.pinv(A)

# v 구하기
v = np.dot(A_inv, b)
v = np.array([round(each, 5) for each in v])
v

array([86.29433, -6.45561])

# 4. 결론

* $a_y$ = 86.29433
* $b_y$ = -6.45561
* dst_road = 16.579  (이미지와 카메라 사이 거리의 폭)
* height_camera = 24.749 (카메라의 높이)

따라서 $s_y, x, y$ 및 카메라의 좌표는 아래와 같다.

$s_y = 86.29433 * p_y -6.45561$  
$x = s_y * p_x$  
$y = s_y * p_y$  
카메라의 좌표 = (0, -16.579, 24.749)